## Fast Charge 

ce notebook montre la dégradation du SoH à cause de la charge rapide.

Conclusion:
- on a 7988 vin parmis eux:
    - 1408 ont fait au moins une charge rapide.
    - 165 l'utilisent plus que la moitié du temps. 
    - 7644 l'utilisent moins d'un quart du temps.

On voit une dégradation du SoH plus élevé pour les vin qui utilisent beaucoup la fast charge contrairement à ceux qui ne l'utilisent pas ou très peu.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import pandas as pd
from core.sql_utils import *

## Function

In [ ]:
def plot_log(df, column):
    def log_function(x, a, b):
        return 1 + a * np.log1p(x/b) 
    fig = go.Figure()
    # create color
    model_colors = {value: px.colors.qualitative.Plotly[i] for i, value in enumerate(df[column].unique())}

    for value in df[column].unique():
        df_model_temp = df[df[column]==value].dropna(subset='soh').sort_values('odometer').copy()
        # fir log function
        popt, _ = curve_fit(log_function, df_model_temp['odometer'], df_model_temp['soh'])
        x_vals = np.linspace(0.1,  df_model_temp.odometer.max(), 500)
        y_vals = log_function(x_vals, *popt)

         # Couleur unique pour le modèle
        color = model_colors[value] 

        # Génération des valeurs ajustées
        fig.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{value}', line=dict(color=color)))
        
    return fig

## data

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b
            on b.id = vm.battery_id
            join fleet f
            on f.id = v.fleet_id
            WHERE f.id not in ('70260bd9-2449-4f5b-81f2-73d9cb6b4b93');"""), con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

In [ ]:
dbeaver_df.columns

In [ ]:
level_charge = dbeaver_df.groupby('vin').agg(
    total_level_1=('level_1', 'sum'),
    total_level_2=('level_2', 'sum'),
    total_level_3=('level_3', 'sum'),
    soh = ('soh', 'last'),
    odometer=('odometer', 'last')
).eval('ratio_level_3=total_level_3/(total_level_3 +  total_level_2+  total_level_1)')

In [ ]:
level_charge["ratio_level_3_cat"] = level_charge['ratio_level_3'].apply(lambda x: "Using fast charging more than half the time" if x >=.5 else 
                                                                        "Using fast charging more than a quarter of the time" if .5> x >.25 
                                                                        else "Low use of fast charging")

## Result 

In [ ]:
level_charge[level_charge['ratio_level_3'] > 0].shape

In [ ]:
level_charge.dropna(subset=['soh', "odometer",])["ratio_level_3_cat"].value_counts()

In [ ]:
fig_fast_charge = plot_log(level_charge, "ratio_level_3_cat")
fig_fast_charge.update_layout(title='Battery degradation depending on the use of fast charging')

In [ ]:
level_charge.isna().sum()

In [ ]:
level_charge.groupby(["ratio_level_3_cat"]).agg(
    soh=('soh', 'mean'),
    total_level_3=('total_level_3', 'mean'),
    odometer=('odometer', 'median')
)


In [ ]:
level_charge_bis = level_charge.dropna(subset=['soh', "odometer",]).copy()

# calcul soh loss pour 100k km
level_charge_bis['soh_loss_pct'] = (1 - level_charge_bis['soh']) * 100

level_charge_bis['soh_loss_per_100k_km'] = level_charge_bis['soh_loss_pct'] / level_charge_bis['odometer'] * 100000

#kWh level 3 / km
level_charge_bis['level3_kWh_per_km'] = level_charge_bis['total_level_3'] / level_charge_bis['odometer']


low_use = level_charge_bis[level_charge_bis['ratio_level_3_cat'] == 'Low use of fast charging']
high_use = level_charge_bis[level_charge_bis['ratio_level_3_cat'].isin(['Using fast charging more than half the time', 'Using fast charging more than a quarter of the time'])]


mean_soh_loss_low = low_use['soh_loss_per_100k_km'].mean()
mean_soh_loss_high = high_use['soh_loss_per_100k_km'].mean()
mean_level3_kWh_per_km = high_use['level3_kWh_per_km'].mean()

level_3_mean_energy_level_3_high_use = high_use['total_level_3'].mean()
level_3_mean_energy_level_3_low_use = low_use['total_level_3'].mean()


delta_soh_per_100k_km = mean_soh_loss_high - mean_soh_loss_low

# SoH perdu par kWh reçu via level 3
soh_loss_per_kWh = delta_soh_per_100k_km / (mean_level3_kWh_per_km * 100000)


print("Energy moyenne de level 3 (low use):", round(level_3_mean_energy_level_3_low_use, 2))
print("Energy moyenne de level 3 (high use):", round(level_3_mean_energy_level_3_high_use, 2))
print("SoH perdu par 100k km (low use):", round(mean_soh_loss_low, 2))
print("SoH perdu par 100k km (high use):", round(mean_soh_loss_high, 2))
print("→ Différence attribuée au Level 3 :", round(delta_soh_per_100k_km, 2))
print("→ SoH perdu par kWh Level 3 :", round(soh_loss_per_kWh, 4), "%/kWh")
